## Matrix Noise Expriment
Automated generation of a similarity matrix for each recipe, adding increasing percentage of matrix noise, iterating over matrix threshold, normalized and not normalized, creating graph, applying community detection algorithms and exporting results to csv file

In [1]:
from data.recipes import small_weak_recipe, small_well_defined_recipe, weak_recipe, well_defined_recipe, equal_recipe,random_recipe, big_weak_recipe, big_well_defined_recipe,big_equal_recipe,big_random_recipe
from classes.compound_generator import CompoundGenerator
from classes.matrix import Matrix
from classes.graph import Graph
import logging
from datetime import datetime
import os
import copy

In [2]:
base_dir = os.path.join("logs")
os.makedirs(base_dir, exist_ok=True)
timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
log_filename = f"/automated_graph_generation_{timestamp}.log"
logging.basicConfig(filename=base_dir+log_filename, level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logging.info("+++++++++++++++++++++++++++++++++++++++++++++++++++")

In [ ]:
recipes = [
    small_weak_recipe, 
    small_well_defined_recipe, 
    weak_recipe, 
    well_defined_recipe, 
    equal_recipe,
    random_recipe, 
    big_weak_recipe, 
    big_well_defined_recipe,
    big_equal_recipe,
    big_random_recipe
]
noise = [True, False]
noise_level = [0.1, 0.5, 0.9]
normalization = [True, False]

def get_recipe_name(recipe):
    name=""
    for y in range(0, len(recipe), 1):
        name += str(recipe[y][1]['name']) + "-" + str(recipe[y][0]) + "-"
    name = name[:-1]
    return name

def log_run(recipe_name, norm_bool, noise_bool, noise_lvl=None):
    print(f"Log {recipe_name}, {norm_bool}, {noise_bool}, {noise_lvl}")
    logging.info(f"Recipe: {recipe_name}")
    logging.info(f"Normalized: {norm_bool}")
    logging.info(f"Noise: {noise_bool}")
    logging.info(f"Noise Level: {noise_lvl}")
    logging.info("------------------------------------------------")
                    
def iterate_treshold(recipe_name, recipe, compounds_list, norm_bool, matrix=None, noise_lvl=None, options=None):
    for x in range(0, 100, 5):
        print(x)
        graph = Graph(recipe, compounds_list, matrix, options)
        graph.apply_matrix_treshold(x) 
        if norm_bool:
            graph.apply_matrix_normalization()
        graph.create_graph()
        # if x > 70:
        #     graph.apply_girvan_newman()
        graph.apply_louvain()
        graph.apply_lpa()
        graph.apply_infomap()
        graph.apply_greedy_modularity()
        graph.calculate_scores()
        graph.export_to_csv(folder_name=recipe_name)
        
       
for recipe in recipes:
    recipe_name = get_recipe_name(recipe)
    generator = CompoundGenerator(recipe, seed=0)
    compounds_list = generator.generate_compound_list()
    original_matrix = Matrix(compounds_list)
    for norm_bool in normalization:
        for noise_bool in noise:
            if noise_bool:
                for noise_lvl in noise_level:
                    original_matrix.add_noise_to_matrix(percentage_to_modify=100, noise_level=noise_lvl)
                    matrix = copy.deepcopy(original_matrix.similarity_matrix)
                    iterate_treshold(recipe_name, recipe, compounds_list, norm_bool, matrix, noise_lvl, original_matrix.options)
                    log_run(recipe_name, norm_bool, noise_bool, noise_lvl)
            else:
                iterate_treshold(recipe_name, recipe, compounds_list, norm_bool, matrix=original_matrix.similarity_matrix, options=original_matrix.options)
                log_run(recipe_name, norm_bool, noise_bool)
                
